<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer_train/notebooks/label_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.3 MB/s 
     |████████████████████████████████| 7.6 MB 31.1 MB/s 
     |████████████████████████████████| 182 kB 90.4 MB/s 


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.4 MB/s 
     |████████████████████████████████| 115 kB 74.0 MB/s 
     |████████████████████████████████| 212 kB 74.2 MB/s 
     |████████████████████████████████| 127 kB 75.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from transformers import pipeline
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

In [5]:
classifier = pipeline("zero-shot-classification", model = 'valhalla/distilbart-mnli-12-1')

In [6]:
dataset = 'cnn_dailymail'
version = '3.0.0'

In [7]:
from datasets import load_dataset

dataset = load_dataset(dataset, version)

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Playing with subsets

In [ ]:
# dataset_subset = dataset['test'].shard(num_shards=100, index=0)

In [ ]:
# output = classifier(dataset_subset['article'], candidate_labels)

In [ ]:
# subset_indices = []

In [ ]:
# for i, article in enumerate(output):
#   if article['labels'][0] == 'about sports' and article['scores'][0]>0.6:
#     subset_indices.append(i)

In [ ]:
# subset_indices

In [ ]:
# a = dataset_subset.select(subset_indices)

In [ ]:
# a.save_to_disk('sports_test.hf')

In [ ]:
# output[0].keys()

# Actual code

In [8]:
def classify_and_subset(dataset, column, classifier, label, threshold, filename):
  '''
  classifies a text into being about the label or not about the label. 
  text is contained in specified column of dataset. 
  threshold is level of certainty required (0-1)
  returns a subsetted dataset
  '''
  subset_indices = []
  # Zero-shot classification requires all the possible labels listed out. 
  labels = [f'about {label}', f'not about {label}']
  for i, article in tqdm(enumerate(classifier(KeyDataset(dataset, column), labels))):
    if article['labels'][0] == labels[0] and article['scores'][0]>threshold:
      subset_indices.append(i)
  subset = dataset.select(subset_indices)
  subset.save_to_disk(filename)

In [9]:
# subset = classify_and_subset(dataset_subset, 'article', classifier, 'sports', 0.6)

In [ ]:
classify_and_subset(dataset['validation'], 'article', classifier, 'sports', 0.6, 'validation_subset')




0it [00:00, ?it/s]


1it [00:28, 28.57s/it]

In [ ]:
# validation_data_new.save_to_disk('cnn_test_sports.hf')

In [ ]:
# from datasets import load_from_disk
# dataset2 = load_from_disk('cnn_test_sports.hf')

Trying cloud - decided against it due to credential issues from colab

In [ ]:
# bucket = client.bucket('news_data_subset')

In [ ]:
# blob=bucket.blob('sports_test.hf/dataset.arrow')

In [ ]:
# blob.upload_from_filename('sports_test.hf/dataset.arrow')

In [ ]:
# from google.cloud import storage

In [ ]:
# def save_to_cloud(dataset, bucket_name, machine_filename, cloud_filename):
#   # dataset.save_to_disk(machine_filename)
#   client = storage.Client()
#   bucket = client.bucket(bucket_name)
#   for i, filename in enumerate(images.keys()):
#     blob = bucket.blob(cloud_filename)
#     blob.upload_from_filename(machine_filename)
#     # # Delete local data after uploading to cloud
#     # if os.path.exists(machine_filename):
#     #     os.remove(machine_filename)

In [ ]:
# save_to_cloud('news_data_subset', 'sports_test.hf', 'sports_test.hf')

In [ ]:
# save_to_cloud('news_data_subset', 'cnn_test_sports.hf', 'cnn_test_sports.hf')